In [35]:
!pip install wandb
import wandb
import wandb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from wandb.keras import WandbCallback

In [36]:
train_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/house-price/train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/house-price/test.csv')
sample_submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/house-price/sample_submission.csv')
train_data.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [37]:
wandb.init(project='House Prices')



In [38]:
train_data = train_data.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'], [0, 1, 2, 3, 4, 5])
train_data = train_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['BrkTil','CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], [0, 1, 2, 3, 4, 5])
train_data = train_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) 
train_data = train_data.replace(['Grvl', 'Pave'], [0, 1])
train_data = train_data.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8])
train_data = train_data.replace(['Gtl','Mod', 'Sev'], [0, 1, 2])
train_data = train_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['Reg','IR1','IR2','IR3'], [0, 1, 2, 3])
train_data = train_data.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'], [0, 1, 2, 3])

In [40]:
train_data = train_data[['SaleCondition','Street', 'BldgType','SaleType','LandSlope','HeatingQC',
                   'Utilities','GarageType','MasVnrType','MSZoning','Foundation','YrSold',
                   'GarageCars','Fireplaces','TotRmsAbvGrd','BedroomAbvGr','KitchenAbvGr',
                   'FullBath','BsmtUnfSF', 'OverallCond', 'LotShape', 'MSSubClass', 'SalePrice']]
                   


In [41]:
test_data = test_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) 
test_data = test_data.replace(['Grvl', 'Pave'], [0, 1])
test_data = test_data.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8])
test_data = test_data.replace(['Gtl','Mod', 'Sev'], [0, 1, 2])
test_data = test_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['Reg','IR1','IR2','IR3'], [0, 1, 2, 3])
test_data = test_data.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'], [0, 1, 2, 3])
test_data = test_data.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'], [0, 1, 2, 3, 4, 5])
test_data = test_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['BrkTil','CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], [0, 1, 2, 3, 4, 5])

In [51]:
test_data = test_data[['SaleCondition','Street', 'BldgType','SaleType','LandSlope','HeatingQC',
                   'Utilities','GarageType','MasVnrType','MSZoning','Foundation','YrSold',
                   'GarageCars','Fireplaces','TotRmsAbvGrd','BedroomAbvGr','KitchenAbvGr',
                   'FullBath','BsmtUnfSF', 'OverallCond', 'LotShape', 'MSSubClass']]
test_data = test_data.fillna(0)
train_data = train_data.fillna(0)

In [64]:
X_train = train_data[['SaleCondition','Street','BldgType','SaleType','LandSlope','HeatingQC',
                   'Utilities','GarageType','MasVnrType','MSZoning','Foundation','YrSold',
                   'GarageCars','Fireplaces','TotRmsAbvGrd','BedroomAbvGr','KitchenAbvGr',
                   'FullBath','BsmtUnfSF', 'OverallCond', 'LotShape', 'MSSubClass']]

Y_train = train_data[['SalePrice']]
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = test_data
Y_test =sample_submission[['SalePrice']]
X_val = np.array(X_val)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)
X_test = np.array(X_test)



In [65]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [66]:
model = tf.keras.models.Sequential([
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='relu'),
])

In [67]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
              loss = tf.keras.losses.mean_squared_logarithmic_error)

In [68]:
model.fit(X_train, Y_train,
                validation_data=(X_val, Y_val),
                epochs=200,
                callbacks=[WandbCallback()]
                )

Epoch 1/200
37/37 [==============================] - 1s 6ms/step - loss: 49.2977 - val_loss: 37.7407
Epoch 2/200
37/37 [==============================] - 0s 2ms/step - loss: 33.3674 - val_loss: 28.6381
Epoch 3/200
37/37 [==============================] - 0s 2ms/step - loss: 25.7345 - val_loss: 22.4054
Epoch 4/200
37/37 [==============================] - 0s 3ms/step - loss: 20.3565 - val_loss: 17.8679
Epoch 5/200
37/37 [==============================] - 0s 3ms/step - loss: 16.2939 - val_loss: 14.1752
Epoch 6/200
37/37 [==============================] - 0s 2ms/step - loss: 12.6280 - val_loss: 10.6191
Epoch 7/200
37/37 [==============================] - 0s 2ms/step - loss: 9.2797 - val_loss: 7.6968
Epoch 8/200
37/37 [==============================] - 0s 2ms/step - loss: 6.7812 - val_loss: 5.6739
Epoch 9/200
37/37 [==============================] - 0s 3ms/step - loss: 5.0606 - val_loss: 4.2861
Epoch 10/200
37/37 [==============================] - 0s 2ms/step - loss: 3.8595 - val_loss: 3.29

In [69]:
print(X_test.shape,Y_test.shape)
model.evaluate(X_test, Y_test)

(1459, 22) (1459, 1)
46/46 [==============================] - 0s 1ms/step - loss: 0.0163


0.016303377225995064